# INPUT

In [101]:
user = 'confluent2'
pwd = 'confluent2'
db_name = 'default'
host = 'localhost'
port = '3307'

In [102]:
table_name = 'ticker_ethusdt'

In [103]:
from sqlalchemy import create_engine, text

# Connect to the database
engine = create_engine(f"mysql+mysqlconnector://{user}:{pwd}@{host}:{port}/{db_name}")

# Test the connection
connection = engine.connect()

In [92]:
connection = engine.connect()

insert_stmt = text(
    f"""
    INSERT INTO {table_name} 
    (closeTime, count, firstId, highPrice, lastId, lastPrice, 
    lowPrice, openPrice, openTime, priceChange, priceChangePercent, 
    quoteVolume, volume, weightedAvgPrice)
    VALUES (
        :closeTime, :count, :firstId, :highPrice, :lastId, :lastPrice, 
        :lowPrice, :openPrice, :openTime, :priceChange, :priceChangePercent, 
        :quoteVolume, :volume, :weightedAvgPrice
    )
    """
)


connection.execute(insert_stmt, data)

# Close the connection
connection.close()

In [ ]:
connection = engine.connect()

insert_stmt = text(
    f"""
    INSERT INTO {table_name} 
        closeTime= :closeTime, 
        count= :count, 
        firstId= :firstId, 
        highPrice= :highPrice, 
        lastId= :lastId, 
        lastPrice= :lastPrice, 
        lowPrice= :lowPrice, 
        openPrice= :openPrice, 
        openTime=:openTime, 
        priceChange= :priceChange, 
        priceChangePercent= :priceChangePercent, 
        quoteVolume= :quoteVolume, 
        volume= :volume, 
        weightedAvgPrice= :weightedAvgPrice

    """
)


connection.execute(insert_stmt, data)

# Close the connection
connection.close()

In [99]:
def db_select(table_name, stmt= 'SELECT * FROM TABLENAME'):
    
    # Test the connection
    connection = engine.connect()
    # Execute a SQL query using text
    query = text(f'SELECT * FROM {table_name}')
    result = connection.execute(query)

    # Fetch the results
    rows = result.fetchall()

    # Close the connection
    connection.close()
    
    return rows
    
    


In [26]:
import requests
def get_data(symbol= 'ETHUSDT', tf= '1m'):
    symbol = symbol.upper()
    endpoint = f'https://api.binance.com/api/v3/ticker?symbol={symbol}&windowSize={tf}'
    res = requests.get(endpoint).json()
    res = {k: float(v) for k, v in res.items() if k != 'symbol'}
    
    return res

data = get_data()
data

{'priceChange': -0.81,
 'priceChangePercent': -0.034,
 'weightedAvgPrice': 2375.35247523,
 'openPrice': 2376.4,
 'highPrice': 2377.55,
 'lowPrice': 2373.6,
 'lastPrice': 2375.59,
 'volume': 802.7666,
 'quoteVolume': 1906853.630344,
 'openTime': 1703862240000.0,
 'closeTime': 1703862322203.0,
 'firstId': 1264844455.0,
 'lastId': 1264845771.0,
 'count': 1317.0}

In [115]:
from sqlalchemy import create_engine, Column, Float, Integer, DateTime, BigInteger
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Connect to the database
engine = create_engine(f"mysql+mysqlconnector://{user}:{pwd}@{host}:{port}/{db_name}")

# ORM Base
Base = declarative_base()

# ORM Model for ticker_ethusdt
class TickerEthusdt(Base):
    __tablename__ = 'ticker_ethusdt'

    id = Column(Integer, primary_key=True, autoincrement=True)
    closeTime = Column(BigInteger, nullable=False)
    count = Column(Float, nullable=False)
    firstId = Column(Float, nullable=False)
    highPrice = Column(Float, nullable=False)
    lastId = Column(Float, nullable=False)
    lastPrice = Column(Float, nullable=False)
    lowPrice = Column(Float, nullable=False)
    openPrice = Column(Float, nullable=False)
    openTime = Column(Integer, nullable=False)
    priceChange = Column(Float, nullable=False)
    priceChangePercent = Column(Float, nullable=False)
    quoteVolume = Column(Float, nullable=False)
    volume = Column(Float, nullable=False)
    weightedAvgPrice = Column(Float, nullable=False)

# Create the table in the database
Base.metadata.create_all(engine)


data = get_data()
# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Insert data into the table using ORM
ticker_entry = TickerEthusdt(**data)
session.add(ticker_entry)
session.commit()

# Close the session
session.close()


C:\Users\kimte\AppData\Local\Temp\ipykernel_18708\1011949344.py:9: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [116]:
db_select(table_name= table_name)

(1703782154046, 2408.0, 1264049116.0, 2382.07, 1264051523.0, 2381.17, 2377.0, 2377.54, 1703782080000.0, 3.63, 0.153, 3463289.547115, 1455.7357, 2379.06478979)
(1703863618858, 1325.0, 1264869869.0, 2356.93, 1264871193.0, 2356.78, 2353.4, 2356.9, 1703863500000.0, -0.12, -0.005, 1046309.163914, 444.2267, 2355.34956344)
(1703865560103, 1297.0, 1264921931.0, 2329.48, 1264923227.0, 2325.84, 2325.6, 2327.06, 1703865480000.0, -1.22, -0.052, 1260725.028813, 541.4607, 2328.37771756)
(1703865768251, 2640.0, 1264925063.0, 2329.59, 1264927702.0, 2320.91, 2320.0, 2329.46, 1703865660000.0, -8.55, -0.367, 2895670.72277, 1246.5414, 2322.96394068)


[(1703782154046, 2408.0, 1264049116.0, 2382.07, 1264051523.0, 2381.17, 2377.0, 2377.54, 1703782080000.0, 3.63, 0.153, 3463289.547115, 1455.7357, 2379.06478979),
 (1703863618858, 1325.0, 1264869869.0, 2356.93, 1264871193.0, 2356.78, 2353.4, 2356.9, 1703863500000.0, -0.12, -0.005, 1046309.163914, 444.2267, 2355.34956344),
 (1703865560103, 1297.0, 1264921931.0, 2329.48, 1264923227.0, 2325.84, 2325.6, 2327.06, 1703865480000.0, -1.22, -0.052, 1260725.028813, 541.4607, 2328.37771756),
 (1703865768251, 2640.0, 1264925063.0, 2329.59, 1264927702.0, 2320.91, 2320.0, 2329.46, 1703865660000.0, -8.55, -0.367, 2895670.72277, 1246.5414, 2322.96394068)]

In [112]:
class Id():
    def __init__(self, name, surname) -> None:
        self.name = name
        self.surname = surname
    
    def combine_name(self):
        self.full_name = self.name +'.'+self.surname
class ShowName(Id):
    def __init__(self, name, surname) -> None:
        super().__init__(name, surname)
        
    def get_fullname(self):
        self.combine_name()
        
    def printName(self):
        print(self.full_name)
        
        
mycls = ShowName('supakorn', 'sae-iab')
# mycls.combine_name()
mycls.get_fullname()
mycls.printName()

supakorn.sae-iab
